In [ ]:
def filtrar_dispositivos(tabela_gases):
  """
    Pergunta ao usuário se ele deseja filtrar os dados de determinados dispositivos e,
    caso sim, solicita os nomes dos dispositivos desejados.

    O usuário deve informar os nomes separados por vírgula, por exemplo:
    "CityIoT1, CityIoT3, etc...".

    Returns:
        list[str] | None: 
            - Lista com os nomes dos dispositivos a serem filtrados, caso o usuário opte por filtrar.
            - None, caso o usuário escolha não aplicar nenhum filtro.
    """
 
  #Limpar a memoria
  dispositivos_para_filtrar = None

  # Loop correção de entrada
  while True:
    try:
      resposta = int(input("Deseja filtrar os dados de determinados dispositivos para construção do gráfico? Digite 1 para sim e 0 para não: "))
      if resposta == 1:
        while True:
          dispositivos_str = input(f"Escreva os dispositivos a serem filtrados ({tabela_gases["Nome_Edge"].dropna().unique().tolist()}). Em caso do multiplos dispositivos, separe-o por vírgula (Ex: CityIoT1,CityIoT6)")
          dispositivos_digitados = [d.strip() for d in dispositivos_str.split(',') if d.strip()]
          dispositivos_invalidos = [d for d in dispositivos_digitados if d not in tabela_gases["Nome_Edge"].dropna().unique().tolist()]
          if dispositivos_invalidos:
            print(f"\n Atenção: os seguintes dispositivos não existem: {dispositivos_invalidos}")
            print("Por favor, digite novamente apenas dispositivos válidos.\n")
            continue
          dispositivos_para_filtrar = dispositivos_digitados
          print(f"Dispositivos a serem filtrados: {dispositivos_para_filtrar}")
          break
        break
      elif resposta == 0:
          print("O gráfico usará todos os dispositivos.")
          break
      else:
          print("Resposta inválida.")

    except ValueError:
      print("Entrada inválida. Por favor, digite um número (1 ou 0).")
  return dispositivos_para_filtrar


def filtrar_datas(tabela_gases):
    """
    Filtra o DataFrame de dados de gases conforme um intervalo de datas informado pelo usuário.

    A função exibe as datas mínima e máxima disponíveis na coluna 'D' e solicita que o usuário
    insira uma data inicial e final no formato "AAAA-MM-DD hh:mm:ss".
    Faz validações de formato, ordem cronológica e compatibilidade de timezone
    antes de retornar os dados filtrados.

    Args:
        tabela_gases (pd.DataFrame):
            DataFrame contendo, no mínimo, a coluna 'D' (datas) e as medições dos sensores.

    Returns:
        pd.DataFrame:
            Um novo DataFrame contendo apenas os registros dentro do intervalo de datas especificado.
    """

    # Verificaçao da existencia da coluna 'D' 
    if 'D' not in tabela_gases.columns:
        raise ValueError("A tabela não contém a coluna 'D' com as datas.")

    # Garante que a coluna esteja no formato datetime com timezone UTC
    if not pd.api.types.is_datetime64_any_dtype(tabela_gases['D']):
        tabela_gases['D'] = pd.to_datetime(tabela_gases['D'], errors='coerce', utc=True)

    # Mostra intervalo de datas disponível
    data_min = tabela_gases['D'].min()
    data_max = tabela_gases['D'].max()

    print("\nDatas disponíveis no conjunto de dados:")
    print(f"Mínima: {data_min}")
    print(f"Máxima: {data_max}\n")

    # Loop de correção de entrada
    while True:
        try:
            data_inicio_str = input("Digite a data inicial (AAAA-MM-DD hh:mm:ss): ")
            data_final_str = input("Digite a data final (AAAA-MM-DD hh:mm:ss): ")

            # Conversão segura para datetime
            data_inicio = pd.to_datetime(data_inicio_str, utc=True, errors='raise')
            data_final = pd.to_datetime(data_final_str, utc=True, errors='raise')

            # Validação de intervalo
            if data_inicio >= data_final:
                print("A data inicial deve ser menor que a data final. Tente novamente.\n")
                continue


            # Se tudo estiver certo, sai do loop
            else:
              break

        except Exception as e:
            print(f"Erro ao processar as datas ({e}). Tente novamente no formato AAAA-MM-DD hh:mm:ss.\n")

    # Aplica o filtro
    tabela_filtrada = tabela_gases[
        (tabela_gases['D'] >= data_inicio) & (tabela_gases['D'] <= data_final)
    ].copy()

    print(f"\nFiltragem concluída. {len(tabela_filtrada)} registros dentro do intervalo escolhido.")
    return tabela_filtrada

In [ ]:
#Novo código com o filtro de dispositivos
import pandas as pd
import plotly as pl
import plotly.express as px
import plotly.graph_objects as go

pd.options.plotting.backend = "plotly"

#Carregando o Dataframe
tabela_gases = pd.read_csv('cityiot.history.csv')

#Modificando as datas para o padrão datatime(Por redundância, foi usado na função filtrar_datas)
tabela_gases['D'] = pd.to_datetime(tabela_gases['D'])

#Chamada da função para filtrar os dispositivos 
dispositivos_selecionados = filtrar_dispositivos(tabela_gases)

if dispositivos_selecionados:
    tabela_gases = tabela_gases[tabela_gases['Nome_Edge'].isin(dispositivos_selecionados)]

#Chamada da função para filtrar as datas
tabela_gases = filtrar_datas(tabela_gases)

#Presonalização do gráfico
grafico = go.Figure()

grafico.add_trace(go.Scatter(x=tabela_gases['D'], y=tabela_gases['TVOC'], mode='lines',name='Concentracao TVOC (ppb) ao longo do tempo',
                             line=dict(color = 'blue', width=3), visible = True))

grafico.add_trace(go.Scatter(x=tabela_gases['D'], y=tabela_gases['G'], mode='lines',name='Concentracao de gases G (ppm) ao longo do tempo',
                             line=dict(color = 'black', width=3), visible = False))


tvoc_faixas = [dict(y0=0, y1=250, line_width=0.5, fillcolor='green', opacity=0.5),
               dict(y0=250, y1=2000, line_width=0.5, fillcolor='yellow', opacity=0.5),
               dict(y0=2000, y1=3000, line_width=0.5, fillcolor='red', opacity=0.5)]

g_faixas = [dict(y0=0, y1=700 , line_width = 0.5, fillcolor='darkgreen', opacity = 0.5),
            dict(y0= 700, y1=1100,line_width = 0.5, fillcolor='lightgreen', opacity = 0.5),
            dict(y0= 1100, y1=1600,line_width = 0.5, fillcolor='yellow', opacity = 0.5),
            dict(y0= 1600, y1=2000,line_width = 0.5, fillcolor='orange', opacity = 0.5),
            dict(y0= 2000, y1=3000,line_width = 0.5, fillcolor='red', opacity = 0.5)]

for r in tvoc_faixas:
  grafico.add_shape(type='rect', xref = 'paper', x0=0, x1=1, **r)


grafico.update_layout(updatemenus=[dict(buttons =[dict(label = 'TVOC', method = 'update', args =[{'visible' :[True, False]},
                  {'title' : 'Concentracao TVOC (ppb) ao longo do tempo',
                   'shapes': [dict(type='rect', xref = 'paper', x0=0, x1=1, **r)for r in tvoc_faixas]}]),
                  dict(label = 'G', method = 'update', args = [{'visible' : [False,True]},
                  {'title' : 'Concentracao G (ppm) ao longo do tempo',
                   'shapes':[dict(type='rect', xref = 'paper', x0=0, x1=1, **r)for r in g_faixas]}] )],
                                        direction = 'down', showactive = True , x=1.15, y= 1.15,
                                        xanchor = 'left', yanchor = 'top')],
                      title = 'Concentracao de gases ao longo do tempo', xaxis_title = 'Data', yaxis_title = 'Concentracao')


grafico.show()


ModuleNotFoundError: No module named 'plotly'